In [15]:
from pathlib import Path
import numpy as np
import pandas as pd
from scipy import interpolate
import matplotlib.pyplot as plt

from pyaw.swarm import save_SW_EFIx_TCT16, save_SW_MAGx_HR_1B

In [1]:
from viresclient import SwarmRequest

In [2]:
request = SwarmRequest()

In [4]:
request.available_collections(details=False)

{'MAG': ['SW_OPER_MAGA_LR_1B',
  'SW_OPER_MAGB_LR_1B',
  'SW_OPER_MAGC_LR_1B',
  'SW_FAST_MAGA_LR_1B',
  'SW_FAST_MAGB_LR_1B',
  'SW_FAST_MAGC_LR_1B'],
 'MAG_HR': ['SW_OPER_MAGA_HR_1B',
  'SW_OPER_MAGB_HR_1B',
  'SW_OPER_MAGC_HR_1B',
  'SW_FAST_MAGA_HR_1B',
  'SW_FAST_MAGB_HR_1B',
  'SW_FAST_MAGC_HR_1B'],
 'EFI': ['SW_OPER_EFIA_LP_1B',
  'SW_OPER_EFIB_LP_1B',
  'SW_OPER_EFIC_LP_1B',
  'SW_FAST_EFIA_LP_1B',
  'SW_FAST_EFIB_LP_1B',
  'SW_FAST_EFIC_LP_1B'],
 'EFI_IDM': ['SW_PREL_EFIAIDM_2_', 'SW_PREL_EFIBIDM_2_', 'SW_PREL_EFICIDM_2_'],
 'EFI_TIE': ['SW_OPER_EFIATIE_2_', 'SW_OPER_EFIBTIE_2_', 'SW_OPER_EFICTIE_2_'],
 'EFI_TCT02': ['SW_EXPT_EFIA_TCT02',
  'SW_EXPT_EFIB_TCT02',
  'SW_EXPT_EFIC_TCT02'],
 'EFI_TCT16': ['SW_EXPT_EFIA_TCT16',
  'SW_EXPT_EFIB_TCT16',
  'SW_EXPT_EFIC_TCT16'],
 'IBI': ['SW_OPER_IBIATMS_2F', 'SW_OPER_IBIBTMS_2F', 'SW_OPER_IBICTMS_2F'],
 'TEC': ['SW_OPER_TECATMS_2F', 'SW_OPER_TECBTMS_2F', 'SW_OPER_TECCTMS_2F'],
 'FAC': ['SW_OPER_FACATMS_2F',
  'SW_OPER_FACBTMS_2F',
  

In [16]:
start_time = '20160311T064700'
end_time = '20160311T064900'
satellite = 'A'
sdir = Path("D:/cleo/master/pyaw/data/swarm/miles2018")
sfn_efi16 = Path("efi16.pkl")
sfn_vfm50 = Path("vfm50.pkl")

In [17]:
save_SW_EFIx_TCT16(start_time, end_time, satellite, sdir,
                   sfn_efi16)
save_SW_MAGx_HR_1B(start_time, end_time, satellite, sdir,
                   sfn_vfm50)

D:\cleo\master\pyaw\data\swarm\miles2018\efi16.pkl already exists, skip save.
D:\cleo\master\pyaw\data\swarm\miles2018\vfm50.pkl already exists, skip save.


In [18]:
df_e = pd.read_pickle(Path(sdir)/Path(sfn_efi16))
df_b = pd.read_pickle(Path(sdir)/Path(sfn_vfm50))
timestamps_e = df_e.index.values

# get rotation matrix

vfm2nec, nec2vfm

In [19]:
VsatN = df_e['VsatN'].values
VsatE = df_e['VsatE'].values
Ehx = df_e['Ehx'].values
Ehy = df_e['Ehy'].values

In [38]:
theta = np.arctan(VsatE/VsatN)
cos_theta = np.cos(theta)
sin_theta = np.sin(theta)
# Stack components to construct the rotation matrices
rotation_matrix = np.array([
    [cos_theta, sin_theta],
    [-sin_theta, cos_theta]
])
# Transpose axes to create a (3, 2, 2) array
rotation_matrix_2d_nec2vfm = np.transpose(rotation_matrix, (2, 0, 1))
rotation_matrix_2d_vfm2nec = rotation_matrix_2d_nec2vfm.transpose(0, 2, 1)
del rotation_matrix

In [39]:
def do_rotation(vectors1,vectors2,rotation_matrix):
    vectors12 = np.stack((vectors1,vectors2),axis=1)
    vectors12_rotated = np.einsum('nij,nj->ni', rotation_matrix, vectors12)
    return vectors12_rotated[:,0],vectors12_rotated[:,1]

# igrf13

In [22]:
latitudes = df_e['Latitude'].values
longitudes = df_e['Longitude'].values
radiuses = df_e['Radius']/1e3  # km

from igrf13 import igrf_utils

igrf_b_nec_n,igrf_b_nec_e,igrf_b_nec_c = igrf_utils.get_igrf13_b_nec(timestamps_e, latitudes,longitudes, radiuses,igrf13shc_path="./igrf13/IGRF13.shc")

the whole loop takes time: 5.298959732055664s


In [23]:
Bz = df_e['Bz'].values
Bz

array([47065.215, 47064.84 , 47064.453, ..., 45743.824, 45742.88 ,
       45741.934], dtype=float32)

In [24]:
igrf_b_nec_c

[47031.73096735575,
 47031.367683847304,
 47030.99772026764,
 47030.63362224372,
 47030.26387734699,
 47029.89925990722,
 47029.52898753066,
 47029.163850417324,
 47028.80326276177,
 47028.43760584882,
 47028.06552717017,
 47027.699349539835,
 47027.32749275109,
 47026.96049799022,
 47026.588112531506,
 47026.22089360885,
 47025.8476822495,
 47025.479942224054,
 47025.10544643134,
 47024.73764246092,
 47024.36291340266,
 47023.993832044434,
 47023.61903078631,
 47023.24942712672,
 47022.874095463776,
 47022.50426764455,
 47022.12764623132,
 47021.757756569554,
 47021.3803043319,
 47021.009131697094,
 47020.632206717426,
 47020.26051075478,
 47019.883054328384,
 47019.510535238114,
 47019.13178344905,
 47018.759803432,
 47018.370002487085,
 47017.996734059285,
 47017.61738051227,
 47017.24358759054,
 47016.87391778876,
 47016.49929955582,
 47016.11811209756,
 47015.74403721075,
 47015.36231524228,
 47014.986645494195,
 47014.60515802139,
 47014.228962327776,
 47013.84694075538,
 47013.4

In [25]:
Bx = df_e['Bx'].values
By = df_e['By'].values

In [26]:
Bx_rotated,By_rotated = do_rotation(Bx,By,rotation_matrix_2d_vfm2nec)

In [27]:
Bx_rotated

array([-3034.2676, -3034.7444, -3035.2295, ..., -3719.7788, -3720.085 ,
       -3720.3853], dtype=float32)

In [28]:
Bx

array([-6025.4023, -6026.9688, -6028.56  , ..., -9247.887 , -9249.308 ,
       -9250.706 ], dtype=float32)

In [31]:
def new_rotation(VsatN,VsatE):
    theta = np.arctan(VsatE/VsatN)
    cos_theta = np.cos(theta)
    sin_theta = np.sin(theta)
    return VsatN*cos_theta + VsatE*sin_theta,-VsatN*sin_theta + VsatE*cos_theta

vx,vy = new_rotation(VsatN,VsatE)

In [33]:
def get_sc2nex(vx,vy):
    theta = np.arctan(VsatE/VsatN)
    cos_theta = np.cos(theta)
    sin_theta = np.sin(theta)
    return vx*cos_theta-vy*sin_theta,vx*sin_theta+vy*cos_theta

get_sc2nex(vx,vy)

(array([-7595.8125, -7595.828 , -7595.8447, ..., -7616.248 , -7616.255 ,
        -7616.2617], dtype=float32),
 array([677.3329 , 677.16394, 676.9922 , ..., 418.94226, 418.83408,
        418.72754], dtype=float32))

In [34]:
VsatN,VsatE

(array([-7595.8125, -7595.8286, -7595.8447, ..., -7616.248 , -7616.255 ,
        -7616.2617], dtype=float32),
 array([677.3329 , 677.16394, 676.99225, ..., 418.94226, 418.83405,
        418.72754], dtype=float32))

In [42]:
vx1,vy1 = do_rotation(VsatN,VsatE,rotation_matrix_2d_nec2vfm)

In [43]:
do_rotation(vx1,vy1,rotation_matrix_2d_nec2vfm)

(array([-7595.8125, -7595.828 , -7595.8447, ..., -7616.248 , -7616.255 ,
        -7616.2617], dtype=float32),
 array([-677.3329 , -677.16394, -676.9922 , ..., -418.94232, -418.834  ,
        -418.72754], dtype=float32))

In [45]:
theta = np.arctan(VsatE/VsatN)
cos_theta = np.cos(theta)
sin_theta = np.sin(theta)

In [47]:
Bx*cos_theta-By*sin_theta

array([-6224.4375, -6225.9956, -6227.5786, ..., -9410.601 , -9412.    ,
       -9413.378 ], dtype=float32)